In [3]:
#install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# Unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

# Install findspark using pip
!pip install -q findspark

# Spark for Python
!pip install pyspark
import findspark
findspark.init("/content/spark-3.0.0-bin-hadoop3.2")
import pyspark

     |████████████████████████████████| 212.3MB 72kB/s 
     |████████████████████████████████| 204kB 52.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=2f627f0785ace15402b087accefcb7cbab776358cba316a7093d1f18bdfb005d
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [4]:

from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Coding_challenge").getOrCreate()

In [6]:
#from pyspark.sql import SparkSession
#spark=SparkSession.builder.appName("work").getOrCreate()

from google.colab import drive
import os.path
from os import path

#######note : if error :file exists occurs , please comment the immediate  below line ie os.mkdir('/content/eluvio_data')

os.mkdir('/content/eluvio_data')

drive.mount('/content/gdrive')

!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive('/content/gdrive/MyDrive/Eluvio_DS_Challenge.rar').extractall('/content/eluvio_data')
eluvio_data='/content/eluvio_data'

Mounted at /content/gdrive
     |████████████████████████████████| 81kB 5.5MB/s 


In [7]:
data=spark.read.csv(eluvio_data,header=True,enforceSchema=True)
Data=data.select(data["up_votes"].cast("int"),data["down_votes"].cast("int"),data["title"],
                data["over_18"].cast("boolean"),data["author"]).na.drop()

Data=Data.withColumn("over_18",Data["over_18"].cast("string"))
Data=Data.withColumn("paper_written",Data["down_votes"]+1)

In [8]:
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import sum
string_index=StringIndexer(inputCol="over_18",outputCol="over_18_index")
Data=string_index.fit(Data).transform(Data)

In [9]:
# lets see which authors got most upvotes

Data_author=Data.groupby("author").sum()
Data_author=Data_author.withColumn("% of likes",(Data_author["sum(up_votes)"]*100)/(Data_author.select(sum("sum(up_votes)")).collect()[0][0]))
Most_up_voted_authors=Data_author.orderBy(Data_author["sum(up_votes)"].desc())

In [10]:
# Most_up_voted_authors (in descending order)
Most_up_voted_authors.show()
Most_up_voted_authors.select("author").head(10)

+----------------+-------------+---------------+------------------+------------------+-------------------+
|          author|sum(up_votes)|sum(down_votes)|sum(paper_written)|sum(over_18_index)|         % of likes|
+----------------+-------------+---------------+------------------+------------------+-------------------+
|     maxwellhill|      1935264|              0|              3846|               0.0|  3.421626723902172|
|       anutensil|      1430816|              0|              4921|               0.0|  2.529741814339961|
|      Libertatea|       832089|              0|              2106|               1.0| 1.4711677368385059|
|   DoremusJessup|       584340|              0|              5032|               0.0| 1.0331372669801098|
|        Wagamaga|       580121|              0|              1490|               0.0|  1.025677900636219|
| NinjaDiscoJesus|       487903|              0|              2431|               1.0| 0.8626326658647303|
|   madazzahatter|       428966|     

[Row(author='maxwellhill'),
 Row(author='anutensil'),
 Row(author='Libertatea'),
 Row(author='DoremusJessup'),
 Row(author='Wagamaga'),
 Row(author='NinjaDiscoJesus'),
 Row(author='madazzahatter'),
 Row(author='madam1'),
 Row(author='kulkke'),
 Row(author='davidreiss666')]

THe top authors with most likes  are : maxwellhill,anutensil,Libertatea,DoremusJessup,Wagamaga,NinjaDiscoJesus,madazzahatter,madam1,kulkke,and davidreiss666


> Indented block



In [11]:
# the authors with least_up votes are( ascending order of upvotes):
Least_up_voted_authors=Data_author.orderBy("sum(up_votes)")
Least_up_voted_authors.show( )

+-----------------+-------------+---------------+------------------+------------------+----------+
|           author|sum(up_votes)|sum(down_votes)|sum(paper_written)|sum(over_18_index)|% of likes|
+-----------------+-------------+---------------+------------------+------------------+----------+
|       luciusXVII|            0|              0|                 1|               0.0|       0.0|
|   postmastercard|            0|              0|                 1|               0.0|       0.0|
|        justwalks|            0|              0|                 1|               0.0|       0.0|
|          debtkid|            0|              0|                 2|               0.0|       0.0|
|           lightP|            0|              0|                 1|               0.0|       0.0|
|          acidity|            0|              0|                 1|               0.0|       0.0|
|      RedditRedIt|            0|              0|                 1|               0.0|       0.0|
|      koh

In [12]:
#Total number of authors who have 0 upvotes and their percentage representation is :
k=Least_up_voted_authors.filter(Least_up_voted_authors["sum(up_votes)"]==0).count()
print(k)
print("Percent of authors have have not got any upvotes = "+ str((k/Least_up_voted_authors.count()) *100 )+"%")

13639
Percent of authors have have not got any upvotes = 16.028910565283816%


In [13]:
upvotes_to_paper_published=Data_author.withColumn("likes_to_paper_ratio",Data_author["sum(up_votes)"]/Data_author["sum(paper_written)"])

13639 authors, ie ( in percentage), 16.028910565283816% of authors have  got 0 upvotes

In [14]:
upvotes_to_paper_published=upvotes_to_paper_published.orderBy(upvotes_to_paper_published["likes_to_paper_ratio"].desc())
upvotes_to_paper_published.columns
upvotes_to_paper_published=upvotes_to_paper_published.select(['author',
 'sum(up_votes)',
 'sum(paper_written)',
 'sum(over_18_index)',
 'likes_to_paper_ratio'
  ,'% of likes'])


# authors with best likes to number of papers published ratio ( descending order)
upvotes_to_paper_published.show()

+-------------------+-------------+------------------+------------------+--------------------+--------------------+
|             author|sum(up_votes)|sum(paper_written)|sum(over_18_index)|likes_to_paper_ratio|          % of likes|
+-------------------+-------------+------------------+------------------+--------------------+--------------------+
|   navysealassulter|        12333|                 1|               0.0|             12333.0|0.021805253642854662|
|          seapiglet|        11288|                 1|               0.0|             11288.0| 0.01995765045978622|
|      DawgsOnTopUGA|        10515|                 1|               0.0|             10515.0|0.018590954516712624|
|Flamo_the_Idiot_Boy|        10289|                 1|               0.0|             10289.0|0.018191377177599256|
| haunted_cheesecake|         9408|                 1|               0.0|              9408.0| 0.01663373277158653|
|bendertheoffender22|         8781|                 1|               0.0

The above data frame represents author with highest up votes per paper. Most of them seems to have written just one article and got large up-votes

In [15]:
#lets see the number of papers that are for adults only

count_not_for_adults=Data.filter(Data["over_18"]=="false").count()
count_for_adults=Data.filter(Data["over_18"]=="true").count()

print("Total number of articles which are for adove 18 audience = " + str(count_for_adults))
print("Total number of articles which are not for adove 18 audience = " + str(count_not_for_adults))

print("% of articles for adults only = " + str((count_for_adults*100)/(count_for_adults+count_not_for_adults)))

Total number of articles which are for adove 18 audience = 315
Total number of articles which are not for adove 18 audience = 499537
% of articles for adults only = 0.06301865352144234


This shows that the data is highly skew in terms of content for above 18 only vs contents for all audience. In fact ,only 0.06% of articles are for above 18 only

In [16]:
from pyspark.sql.functions import length
Data=Data.withColumn("length_of_title",length(Data["title"]))
Data_to_be_used=Data.select(["title","over_18_index","length_of_title"])
Data_to_be_used.show()

+--------------------+-------------+---------------+
|               title|over_18_index|length_of_title|
+--------------------+-------------+---------------+
|Scores killed in ...|          0.0|             33|
|Japan resumes ref...|          0.0|             32|
|US presses Egypt ...|          0.0|             31|
|Jump-start econom...|          0.0|             44|
|Council of Europe...|          0.0|             47|
|Hay presto! Farme...|          0.0|             97|
|Strikes, Protests...|          0.0|             59|
|The U.N. Mismanag...|          0.0|             30|
|Nicolas Sarkozy t...|          0.0|             41|
|US plans for miss...|          0.0|             71|
|Archbishop of Can...|          0.0|             95|
|Top US Envoy: Vio...|          0.0|             53|
|Team building flo...|          0.0|             90|
|Migrant workers t...|          0.0|             60|
| Sarkozy, Girlfri...|          0.0|             40|
|Nicolas Sarkozy, ...|          0.0|          

In [17]:
# this shows thar irrespective of category of article, the average lengh of its title is almost the same
Data_to_be_used.groupBy("over_18_index").mean().show()

+-------------+------------------+--------------------+
|over_18_index|avg(over_18_index)|avg(length_of_title)|
+-------------+------------------+--------------------+
|          0.0|               0.0|   88.76956862054263|
|          1.0|               1.0|   87.62222222222222|
+-------------+------------------+--------------------+



However, length of title doesnt give any insights about classifying data over_18 and under_18

In [18]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF,VectorAssembler
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.classification import RandomForestClassifier,NaiveBayes

tokenizers= Tokenizer(inputCol="title",outputCol="tokens")
stop_words_removed=StopWordsRemover(inputCol="tokens",outputCol="stop_words_removed")
vector_count=CountVectorizer(inputCol="stop_words_removed",outputCol="vector_c")
inverse_df=IDF(inputCol="vector_c",outputCol="idf")
ready_data=VectorAssembler(inputCols=["idf","length_of_title"],outputCol="feature")
naive_bias=NaiveBayes(featuresCol="feature",labelCol="over_18_index")

data_to_process=Pipeline(stages=[tokenizers,stop_words_removed,vector_count,inverse_df,ready_data])
process=data_to_process.fit(Data_to_be_used)
final_data=process.transform(Data_to_be_used)
train,test=final_data.randomSplit([0.8,0.2])

train.printSchema()
train.show()

root
 |-- title: string (nullable = true)
 |-- over_18_index: double (nullable = false)
 |-- length_of_title: integer (nullable = true)
 |-- tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- stop_words_removed: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- vector_c: vector (nullable = true)
 |-- idf: vector (nullable = true)
 |-- feature: vector (nullable = true)

+--------------------+-------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|over_18_index|length_of_title|              tokens|  stop_words_removed|            vector_c|                 idf|             feature|
+--------------------+-------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|	 Chevron admits ...|          0.0|            214|[, , chevron, adm...|[, , chevron, adm...

In [83]:
model=naive_bias.fit(train)
finale=model.transform(test)

from pyspark.ml.evaluation import BinaryClassificationEvaluator

# AS ITS A SCREW DATA SET, ACCURACY IS NOT THE BEST METRIC,SO HERE WE ARE USING AREA UNDER CURVE

binary_Eval=BinaryClassificationEvaluator(labelCol="over_18_index",rawPredictionCol="rawPrediction")

print("Area under Curve=" + str(binary_Eval.evaluate(finale)*100))

Area under Curve=41.63591163612101


In the above code, though , naive bias seems fast, its area under curve is very low (42 % approx). So, we will explore other models

**In the following code, we tried to generate a RNN model to classify titles as for all people vs for people above 18 only . Though its really tough to do for because of highly skew data, we tried to have a vague idea of time taken and performance achieved**

In [84]:
import csv
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words=stopwords.words('english')

pseudo_label,sent=[],[]
with open("/content/eluvio_data/Eluvio_DS_Challenge.csv", 'r') as csvfile:
  reader = csv.reader(csvfile, delimiter=',')
  for row in reader:
        pseudo_label.append(row[5])
        sentence=row[4]
        for word in stop_words:
          word_to_be_replaced=" " + word + " "
          sentence=sentence.replace(word_to_be_replaced, " ")
        sent.append(sentence)
  
label=[0 if pseudo_label[i]=="False" else 1 for i in range(len(pseudo_label))]

#print(len(sent))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


We have 509237 sentences. So even if we take 95% of the data in train and just 5% in test, we have len(train),len(test)=483775,25461 cases. We will be using this split

In [85]:
training_size=int(0.95*len(sent))
training_labels,training_sentences=label[:training_size],sent[:training_size]
testing_labels,testing_sentences=label[training_size:],sent[training_size:]


In [86]:
import tensorflow 
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer( oov_token="@%^&")
tokenizer.fit_on_texts(training_sentences)
total_words_used=len(tokenizer.word_counts)
print(total_words_used)

reverse_sorted_words = sorted(tokenizer.word_index, key=tokenizer.word_index.get, reverse=False)


# now,lets not take all words in this corpos, we will take only  80,000 most repeated words ( based on intrusion)
tokenizer = Tokenizer( oov_token="@%^&",num_words=80000)
tokenizer.fit_on_texts(training_sentences)

training_sentences_to_tokens=tokenizer.texts_to_sequences(training_sentences)

# now,we will padd these generted sentences_to_token and make sure that they are of fixed length. As the length of largest statement is 300, however, the average ,
#length is only 76 with standard deviation of 46.  We will make every padded statement to be of size of 120 and we will add 0 in "pre" format, ie ,for
# smaller statements, they will start with 0 to make the total length of sentence to 120

padded_train_statements=pad_sequences(sequences=training_sentences_to_tokens,maxlen=120,padding="pre")

train_x=padded_train_statements
train_y=np.asarray(training_labels)

test=tokenizer.texts_to_sequences(testing_sentences)
test_x=pad_sequences(sequences=test,maxlen=120,padding="pre")
test_y=np.asarray(testing_labels)


103788


So, we can see that ,exlcuding commonly used stop words, this data used about 103788 distint wordsand the top 100 most common words are :says',
 'china',
 'us',
 'new',
 'u',
 's',
 'syria',
 'russia',
 'world',
 'police',
 'government',
 'israel',
 'iran',
 'a',
 'president',
 'killed',
 'people',
 'state',
 'attack',
 'in',
 'war',
 'russian',
 'military',
 'uk',
 'north',
 'year',
 'south',
 'korea',
 'minister',
 'news',
 'said',
 'two',
 'first',
 'ukraine',
 'india',
 '000',
 'nuclear',
 'syrian',
 'un',
 'years',
 'israeli',
 'one',
 'al',
 'chinese',
 'isis',
 'court',
 'country',
 'death',
 'eu',
 'iraq',
 'egypt',
 'japan',
 'dead',
 'may',
 'pakistan',
 'could',
 'report',
 'anti',
 'leader',
 'man',
 'islamic',
 'say',
 'turkey',
 'security',
 'to',
 'forces',
 'saudi',
 'is',
 'found',
 'attacks',
 '1',
 'gaza',
 'oil',
 'british',
 'crisis',
 'group',
 'rights',
 'french',
 'deal',
 'million',
 'obama',
 '2',
 'city',
 'france',
 'air',
 'former',
 'army',
 'it',
 'officials',
 'women',
 'germany',
 'talks',
 'europe',
 'foreign',
 'international',
 'rebels',
 'putin',
 'human'

In [87]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,GlobalAveragePooling1D,Conv1D,LSTM,Bidirectional,Flatten


model=Sequential()
model.add(Embedding(input_dim=80000,output_dim=100,input_length=120))
model.add(Bidirectional(LSTM(64,return_sequences=True)))
#model.add(Bidirectional(LSTM(64,return_sequences=True)))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(128,activation="relu"))
model.add(Dense(256,activation="relu"))
model.add(Dense(1,activation="sigmoid"))
#model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(train_x, train_y, epochs=10, validation_data=(test_x, test_y), verbose=1,batch_size=1024)

pred = model.predict(test_x, batch_size=32, verbose=1)
predicted = np.argmax(pred, axis=1)

from sklearn.metrics import f1_score

corrected_prediction=[]
for i in predicted:
  if i ==0:
    corrected_prediction.append(0)
  else:
    corrected_prediction.append(1)

corrected_prediction=np.asarray(corrected_prediction).reshape(-1,1)

test_y=test_y.reshape(-1,1)
print("the recall obtained ="+ " " + str((f1_score(test_y,predicted,average="micro")*100)))


Epoch 1/10
473/473 [==============================] - 103s 209ms/step - loss: 0.0441 - accuracy: 0.9994 - val_loss: 0.0059 - val_accuracy: 0.9993
Epoch 2/10
473/473 [==============================] - 97s 204ms/step - loss: 0.0035 - accuracy: 0.9996 - val_loss: 0.0051 - val_accuracy: 0.9994
Epoch 3/10
473/473 [==============================] - 97s 205ms/step - loss: 0.0018 - accuracy: 0.9997 - val_loss: 0.0072 - val_accuracy: 0.9994
Epoch 4/10
473/473 [==============================] - 97s 205ms/step - loss: 5.5758e-04 - accuracy: 0.9999 - val_loss: 0.0079 - val_accuracy: 0.9994
Epoch 5/10
473/473 [==============================] - 97s 205ms/step - loss: 2.9269e-04 - accuracy: 0.9999 - val_loss: 0.0104 - val_accuracy: 0.9993
Epoch 6/10
473/473 [==============================] - 97s 205ms/step - loss: 2.4193e-04 - accuracy: 0.9999 - val_loss: 0.0102 - val_accuracy: 0.9993
Epoch 7/10
473/473 [==============================] - 97s 205ms/step - loss: 1.1298e-04 - accuracy: 1.0000 - val_loss

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

random_forest=RandomForestClassifier(featuresCol="feature",labelCol="over_18_index",seed=1,numTrees=50)

model_random_forest=random_forest.fit(train)
finale_random_forest=model_random_forest.transform(test)

binary_Eval=BinaryClassificationEvaluator(labelCol="over_18_index",rawPredictionCol="rawPrediction")

print("Area under Curve=" + str(binary_Eval.evaluate(finale_random_forest)*100))



While using random forest, the area under curve was 71.229%, however, random forest takes lot of time. 

In the following last section, we tried to make a model that automatically generats text after being trained on litrature style of a person. We took the author with highest numbers of titles and trained a model using his/her title. Then we gave model few random words and made model to generate sentences in a way, similar to writing style of author.

The top author is "davidreiss666" and a generated 10 word sentence is :

Habyarimana commander lead Ireland  airlines state media says central jail where enforcement to show eu coalition far in its peers of its gdp is


In [20]:
import tensorflow 
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

author_with_most_papers=Data_author.orderBy(Data_author["sum(paper_written)"].desc())
most_written_author=author_with_most_papers.head(1)[0][0]
davidreiss666=Data.filter(Data["Author"]==most_written_author)
#davidreiss666.show()

titles_of_top_author=davidreiss666.select(["title"]).collect()
text_obtained_from_top_authors_articles=[]
for i in range(len(titles_of_top_author)):
  text_obtained_from_top_authors_articles.append(titles_of_top_author[i][0]) 


tokens_for_top_authors=Tokenizer(oov_token="@!@#@$$%$",lower=True)
tokens_for_top_authors.fit_on_texts(text_obtained_from_top_authors_articles)

to_token_conversations_of_sentence=[]
for i in text_obtained_from_top_authors_articles:
  s=tokens_for_top_authors.texts_to_sequences([i])[0]
  #print(s)
  for j in range(1,len(s)):
    n_gram=s[:j+1]
    to_token_conversations_of_sentence.append(n_gram)

max_length=max([len(i) for i in to_token_conversations_of_sentence])

total_words=len(tokens_for_top_authors.word_index)+1


padding_tokens_of_top_authors=pad_sequences(to_token_conversations_of_sentence,maxlen=55,padding="pre")

training_for_top_author_data,labels_for_top_authors=padding_tokens_of_top_authors[:,:-1],padding_tokens_of_top_authors[:,-1]

labels_for_top_authors=labels_for_top_authors.reshape(-1,1)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,GlobalAveragePooling1D,Conv1D,LSTM,Bidirectional,Flatten

model = Sequential()
model.add(Embedding(total_words, 80, input_length=max_length-1))
model.add(Bidirectional(LSTM(20)))
model.add(Dense(128,activation="tanh"))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

model.fit(training_for_top_author_data,labels_for_top_authors,batch_size=512,verbose=1,epochs=700)

text_obtained_from_top_authors_articles

text="Habyarimana commander lead Ireland "

new_title_length=20

for i in range(new_title_length):
  token=pad_sequences([tokens_for_top_authors.texts_to_sequences([text])[0]],maxlen=54,padding="pre")

  result=int(model.predict_classes(token,verbose=0))

  generated_word=" "

  if result in tokens_for_top_authors.index_word.keys():
    generated_word=tokens_for_top_authors.index_word[result]


  text=text+ " " + generated_word

print(text)
  




Epoch 1/700
440/440 [==============================] - 57s 48ms/step - loss: 8.0716 - accuracy: 0.0356
Epoch 2/700
440/440 [==============================] - 21s 48ms/step - loss: 7.1338 - accuracy: 0.0588
Epoch 3/700
440/440 [==============================] - 21s 47ms/step - loss: 6.8368 - accuracy: 0.0823
Epoch 4/700
440/440 [==============================] - 22s 51ms/step - loss: 6.5378 - accuracy: 0.1066
Epoch 5/700
440/440 [==============================] - 22s 49ms/step - loss: 6.2957 - accuracy: 0.1224
Epoch 6/700
440/440 [==============================] - 22s 49ms/step - loss: 6.1278 - accuracy: 0.1321
Epoch 7/700
440/440 [==============================] - 22s 49ms/step - loss: 5.9695 - accuracy: 0.1429
Epoch 8/700
440/440 [==============================] - 22s 49ms/step - loss: 5.8284 - accuracy: 0.1528
Epoch 9/700
440/440 [==============================] - 22s 50ms/step - loss: 5.6904 - accuracy: 0.1612
Epoch 10/700
440/440 [==============================] - 23s 52ms/step - l

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Habyarimana commander lead Ireland  s family s soldiers sentenced on anti mafia city appears prosecutor said should be held to his presidential election chatting
